# Mass spectrometry data

The objective of this exercise is to read in raw peptide MSMS spectrum information and output a dataframe.
The .msp file can be downloaded [here](ftp://chemdata.nist.gov/download/peptide_library/libraries/cptaclib/2015/cptac2_mouse_hcd_selected.msp.tar.gz).

The information in this ASCII based text file is organized spectrum by spectrum.
The first line per spectrum provides formatted like this:

&emsp;<code>Name: sequence/charge_nmods_collisionenergy</code>

followed by a comment section wich can be disregarded and the actual spectrum data which is tab-separated:

&emsp;<code>m/z&emsp;intensity&emsp;additional_info</code>

Spectra are separated by an empty line.

Code a function that returns two DataFrames or arrays containing the processed and filtered data. The first one should contain the spectrum information (n_spectra, n_m/z_features) and the second one the sequences per row (n_spectra).

Here are some general guidelines:

* The m/z values need to be binned to integer values (mathematically rounded), otherwise the dataframe size would get out of hand. This will allow for multiple values mapped to a single bin (e.g. if there are peaks at 145.1 and 145.2). Here, only the maximum of those peaks should be kept in the final dataframe.

* Rows that are all-zero should be dropped.

Your function should allow for selecting a range on the x-axis (m/z-range). All peaks outside of this range can be disregarded. Furthermore, only spectra within a set collision energy range and a maximum sequence length should be contained in the output dataframe.

The faster your function runs, the better. I will time them all in the end.

In [2]:
import numpy as np
import pandas as pd
import timeit
import re
import io

In [19]:
def msp_to_df(
    input_file,
    max_seq_len=30,
    min_ce=36,
    max_ce=40,
    mz_min=135,
    mz_max=1400,
):
    """
    Function to read spectrum data from .msp file and convert to dataframe.
    Args:
        input_file (str): path to .msp file
        max_seq_len (int): maximum acceptable sequence length
        min_ce (int): minimum collision energy of spectra to be included in df
        max_ce (int): maximum collision energy of spectra to be included in df
        mz_min (int): lower boundary for m/z to be included in df
        mz_max (int): lower boundary for m/z to be included in df

    Returns:
        df (pd.DataFrame or np.array): spectrum information within defined parameters [n_spectra, n_features]
        seqs (pd.DataFrame or np.array): sequences
    """ 
    df = pd.DataFrame()
    seqs = list()
    
    msp_file = open(input_file, "r")
    msp_data = msp_file.read().split("\n\n")
    msp_data.pop()
    for spec in msp_data:
   
        sequence = re.search(r' (.*?)/', spec).group(1)
        energy = re.search(r'[+-]?([0-9]*[.])?[0-9]+(?=eV)',spec).group()

        
        if len(sequence) < max_seq_len and float(energy) > min_ce and float(energy) < max_ce:
            data = pd.read_csv(io.StringIO(spec), header = None, sep = "\t", skiprows = 4, names = ["m/z", "intensity", "comment"])
            data = data[["m/z", "intensity"]]
            data["m/z"] = pd.to_numeric(data["m/z"]).round(0)
            data = data[(data["m/z"] > mz_min)  & (data["m/z"] < mz_max)]
            data = data.groupby("m/z")["intensity"].max()
            df = df.append(data.T, ignore_index = True) 
            seqs.append(sequence)

        
        
        

        #peaks = re.search(r': (.*?)$',line).group(1)
            

    

    seqs = pd.DataFrame(seqs)   
    return df, seqs